# DSAI HW1 Peak Load Forecasting
請根據台電歷史資料，預測未來七天的"電力尖峰負載"(MW)

In [6]:
import numpy as np
import pandas as pd
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dense

/Users/Solitude6060/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/Solitude6060/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## Data Preprocessing

Read data and parser - 
將CSV資料讀入後以逗號拆開，只取包含時間前4個feature

In [7]:
def readfile(path):
    file = open(path, "r+")
    f = file.readlines()
    return_tp_list = []
    test_list = []
    for i in f[1:366]:
        #print(i)
        day_list = i[:-1].split(",")
        temp_list = []
        for d in day_list[:2]:
            temp_list.append(d)
        for d in day_list[3:5]:
            temp_list.append(d)
        test_list.append(day_list[2])
        #print(temp_list)
        #print(len(temp_list))
        return_tp_list.append(temp_list)
    
    return return_tp_list, test_list

取2017和2018年的資料

In [8]:
tp_2017, target_2017_list = readfile('data/taipower_2017.csv')
tp_2018, target_2018_list = readfile('data/taipower_2018.csv')

#print(tp_2018)

Parser data ，取去年同一時間和上一週的資料

In [12]:
def sliceData(tp_data_2017, tp_data_2018):
    #index = 1
    con_list = []
    data_2017_list = []
    for t in range(len(tp_data_2017[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2017[t][1:]
        else:
            #if t is 0:
            con_list += tp_data_2017[t][1:]
            data_2017_list.append(con_list)
            con_list = []
    
    con_list = []
    data_2018_list = []
    for t in range(len(tp_data_2018[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2018[t][1:]
            #print(con_list)
        else:
            #if t is 0:
            con_list += tp_data_2018[t][1:]    
            data_2018_list.append(con_list)
            con_list = []
    
    #print("2017", data_2017_list[0])
    #print("2018", data_2018_list[0])
    train_list = []
    temp_list =[]
    for i in range(len(data_2017_list)):
        if i == 0:
            temp_list = data_2017_list[i] + data_2017_list[-1]
            #print("i = 0", temp_list)
        else:
            temp_list = data_2017_list[i] + data_2017_list[i]
            #print("i != 0", temp_list)
        train_list.append(temp_list)
    
    return train_list

In [13]:
train_X = sliceData(tp_2017, tp_2018)
#print(train_X[0])
#print(len(train_X[0]))

取得Training target，也就是training data中實際的尖峰負載預測值

In [14]:
def getTrainY(target_list):
    train_y_list = []
    con_list = []
    for i in range(len(target_list[:-1])):
        if (i+1) % 7 is not 0:
            con_list.append(target_list[i])
        else:
            con_list.append(target_list[i])
            train_y_list.append(con_list)
            con_list = []
    return train_y_list

In [15]:
train_Y = getTrainY(target_2018_list)
np_train_X = np.array(train_X)
np_train_Y = np.array(train_Y)

# Bulid Model
以Keras開發，用NN概念實現Linear regression
以多層Dense，最後一層output 1x7 向量代表整週的預測

In [17]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(x_shape[1], ), activation='linear'))
    model.add(Dense(units=7,activation='linear'))
    model.summary()
    return model

# Loss function
root_mean_squared_error

In [18]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [19]:
LinearModel = bulidModel(np_train_X.shape)
LinearModel.compile(loss=root_mean_squared_error, optimizer='rmsprop')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               11008     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 1799      
Total params: 12,807
Trainable params: 12,807
Non-trainable params: 0
_________________________________________________________________


In [20]:
train_history = LinearModel.fit(np_train_X , np_train_Y , batch_size=128 , epochs=1000 , verbose=1, validation_split=0.2)

Train on 41 samples, validate on 11 samples
Epoch 1/1000
41/41 [==============================] - 0s 6ms/step - loss: 28164.1621 - val_loss: 10340.5039
Epoch 2/1000
41/41 [==============================] - 0s 102us/step - loss: 11507.5713 - val_loss: 1820.3417
Epoch 3/1000
41/41 [==============================] - 0s 125us/step - loss: 2170.8840 - val_loss: 6095.0073
Epoch 4/1000
41/41 [==============================] - 0s 176us/step - loss: 7055.3218 - val_loss: 4643.3184
Epoch 5/1000
41/41 [==============================] - 0s 107us/step - loss: 4899.0991 - val_loss: 5338.9878
Epoch 6/1000
41/41 [==============================] - 0s 142us/step - loss: 6044.9292 - val_loss: 3984.8708
Epoch 7/1000
41/41 [==============================] - 0s 187us/step - loss: 4511.6919 - val_loss: 4687.4482
Epoch 8/1000
41/41 [==============================] - 0s 151us/step - loss: 5151.1035 - val_loss: 3583.0647
Epoch 9/1000
41/41 [==============================] - 0s 208us/step - loss: 4296.5811 - val

Epoch 76/1000
41/41 [==============================] - 0s 128us/step - loss: 3466.2812 - val_loss: 2677.3408
Epoch 77/1000
41/41 [==============================] - 0s 95us/step - loss: 3380.9553 - val_loss: 3046.6274
Epoch 78/1000
41/41 [==============================] - 0s 116us/step - loss: 3384.2122 - val_loss: 2532.7529
Epoch 79/1000
41/41 [==============================] - 0s 110us/step - loss: 3295.6338 - val_loss: 3066.3240
Epoch 80/1000
41/41 [==============================] - 0s 117us/step - loss: 3377.9260 - val_loss: 2653.3660
Epoch 81/1000
41/41 [==============================] - 0s 102us/step - loss: 3390.2478 - val_loss: 3142.1697
Epoch 82/1000
41/41 [==============================] - 0s 117us/step - loss: 3522.9214 - val_loss: 2911.9734
Epoch 83/1000
41/41 [==============================] - 0s 129us/step - loss: 3474.6887 - val_loss: 3111.3281
Epoch 84/1000
41/41 [==============================] - 0s 124us/step - loss: 3662.0828 - val_loss: 3215.0015
Epoch 85/1000
41/41 

Epoch 151/1000
41/41 [==============================] - 0s 133us/step - loss: 3325.0332 - val_loss: 2545.2959
Epoch 152/1000
41/41 [==============================] - 0s 129us/step - loss: 3305.8528 - val_loss: 3346.3672
Epoch 153/1000
41/41 [==============================] - 0s 123us/step - loss: 3400.8899 - val_loss: 2408.4482
Epoch 154/1000
41/41 [==============================] - 0s 150us/step - loss: 3318.7439 - val_loss: 3265.2283
Epoch 155/1000
41/41 [==============================] - 0s 163us/step - loss: 3220.3433 - val_loss: 2310.7295
Epoch 156/1000
41/41 [==============================] - 0s 165us/step - loss: 3224.0229 - val_loss: 3128.0271
Epoch 157/1000
41/41 [==============================] - 0s 165us/step - loss: 3162.0461 - val_loss: 2512.0383
Epoch 158/1000
41/41 [==============================] - 0s 145us/step - loss: 3293.6411 - val_loss: 2990.1482
Epoch 159/1000
41/41 [==============================] - 0s 140us/step - loss: 3206.4753 - val_loss: 2694.9595
Epoch 160/

Epoch 226/1000
41/41 [==============================] - 0s 124us/step - loss: 3178.0388 - val_loss: 3063.4031
Epoch 227/1000
41/41 [==============================] - 0s 100us/step - loss: 3272.2703 - val_loss: 2640.2966
Epoch 228/1000
41/41 [==============================] - 0s 103us/step - loss: 3420.3933 - val_loss: 3545.4893
Epoch 229/1000
41/41 [==============================] - 0s 111us/step - loss: 3347.5007 - val_loss: 2384.6228
Epoch 230/1000
41/41 [==============================] - 0s 109us/step - loss: 3423.9673 - val_loss: 3420.2341
Epoch 231/1000
41/41 [==============================] - 0s 96us/step - loss: 3122.9336 - val_loss: 2163.2822
Epoch 232/1000
41/41 [==============================] - 0s 128us/step - loss: 3237.5698 - val_loss: 3167.5085
Epoch 233/1000
41/41 [==============================] - 0s 117us/step - loss: 2974.0950 - val_loss: 2218.4377
Epoch 234/1000
41/41 [==============================] - 0s 174us/step - loss: 3107.0222 - val_loss: 2888.2671
Epoch 235/1

Epoch 301/1000
41/41 [==============================] - 0s 137us/step - loss: 3104.5542 - val_loss: 3599.3103
Epoch 302/1000
41/41 [==============================] - 0s 162us/step - loss: 3432.4600 - val_loss: 2123.9521
Epoch 303/1000
41/41 [==============================] - 0s 140us/step - loss: 3240.6257 - val_loss: 3558.5115
Epoch 304/1000
41/41 [==============================] - 0s 157us/step - loss: 3206.3347 - val_loss: 1945.9836
Epoch 305/1000
41/41 [==============================] - 0s 138us/step - loss: 3127.9973 - val_loss: 3321.3721
Epoch 306/1000
41/41 [==============================] - 0s 131us/step - loss: 3041.5830 - val_loss: 2163.4729
Epoch 307/1000
41/41 [==============================] - 0s 135us/step - loss: 3115.8972 - val_loss: 3081.5234
Epoch 308/1000
41/41 [==============================] - 0s 132us/step - loss: 3097.6323 - val_loss: 2538.5876
Epoch 309/1000
41/41 [==============================] - 0s 116us/step - loss: 3108.5876 - val_loss: 2765.3186
Epoch 310/

...

Epoch 676/1000
41/41 [==============================] - 0s 117us/step - loss: 2863.2422 - val_loss: 3016.3682
Epoch 677/1000
41/41 [==============================] - 0s 164us/step - loss: 2589.0474 - val_loss: 1956.2997
Epoch 678/1000
41/41 [==============================] - 0s 157us/step - loss: 2805.5581 - val_loss: 2526.7576
Epoch 679/1000
41/41 [==============================] - 0s 136us/step - loss: 2424.8564 - val_loss: 2051.2014
Epoch 680/1000
41/41 [==============================] - 0s 121us/step - loss: 2562.3206 - val_loss: 2215.9836
Epoch 681/1000
41/41 [==============================] - 0s 127us/step - loss: 2455.0103 - val_loss: 2414.5693
Epoch 682/1000
41/41 [==============================] - 0s 141us/step - loss: 2577.6599 - val_loss: 2087.2292
Epoch 683/1000
41/41 [==============================] - 0s 148us/step - loss: 2630.0872 - val_loss: 2543.6748
Epoch 684/1000
41/41 [==============================] - 0s 130us/step - loss: 2576.2512 - val_loss: 2127.3708
Epoch 685/

Epoch 751/1000
41/41 [==============================] - 0s 132us/step - loss: 2590.3950 - val_loss: 2662.2029
Epoch 752/1000
41/41 [==============================] - 0s 155us/step - loss: 2406.8967 - val_loss: 1743.6464
Epoch 753/1000
41/41 [==============================] - 0s 105us/step - loss: 2550.0391 - val_loss: 2419.2510
Epoch 754/1000
41/41 [==============================] - 0s 145us/step - loss: 2362.8875 - val_loss: 1896.3665
Epoch 755/1000
41/41 [==============================] - 0s 111us/step - loss: 2466.7158 - val_loss: 2206.5876
Epoch 756/1000
41/41 [==============================] - 0s 155us/step - loss: 2334.5046 - val_loss: 2042.9396
Epoch 757/1000
41/41 [==============================] - 0s 143us/step - loss: 2411.3267 - val_loss: 2027.3179
Epoch 758/1000
41/41 [==============================] - 0s 161us/step - loss: 2334.0789 - val_loss: 2239.6423
Epoch 759/1000
41/41 [==============================] - 0s 131us/step - loss: 2426.6504 - val_loss: 1936.4519
Epoch 760/

Epoch 826/1000
41/41 [==============================] - 0s 167us/step - loss: 2618.0640 - val_loss: 2701.6375
Epoch 827/1000
41/41 [==============================] - 0s 131us/step - loss: 2386.1008 - val_loss: 1634.6583
Epoch 828/1000
41/41 [==============================] - 0s 131us/step - loss: 2592.3628 - val_loss: 2717.0925
Epoch 829/1000
41/41 [==============================] - 0s 141us/step - loss: 2329.7488 - val_loss: 1548.9719
Epoch 830/1000
41/41 [==============================] - 0s 123us/step - loss: 2616.2356 - val_loss: 2830.1970
Epoch 831/1000
41/41 [==============================] - 0s 128us/step - loss: 2437.5593 - val_loss: 1574.0312
Epoch 832/1000
41/41 [==============================] - 0s 153us/step - loss: 2569.5552 - val_loss: 2528.6094
Epoch 833/1000
41/41 [==============================] - 0s 137us/step - loss: 2312.6309 - val_loss: 1731.1621
Epoch 834/1000
41/41 [==============================] - 0s 147us/step - loss: 2422.7861 - val_loss: 2260.2607
Epoch 835/

Epoch 901/1000
41/41 [==============================] - 0s 117us/step - loss: 2338.7864 - val_loss: 1353.7467
Epoch 902/1000
41/41 [==============================] - 0s 131us/step - loss: 2515.4478 - val_loss: 3067.0273
Epoch 903/1000
41/41 [==============================] - 0s 139us/step - loss: 2497.6062 - val_loss: 1569.5337
Epoch 904/1000
41/41 [==============================] - 0s 168us/step - loss: 2704.3933 - val_loss: 2680.6819
Epoch 905/1000
41/41 [==============================] - 0s 126us/step - loss: 2322.9170 - val_loss: 1697.3104
Epoch 906/1000
41/41 [==============================] - 0s 133us/step - loss: 2485.8623 - val_loss: 2218.1475
Epoch 907/1000
41/41 [==============================] - 0s 147us/step - loss: 2266.3171 - val_loss: 2187.3757
Epoch 908/1000
41/41 [==============================] - 0s 129us/step - loss: 2426.4429 - val_loss: 1779.5907
Epoch 909/1000
41/41 [==============================] - 0s 118us/step - loss: 2367.0891 - val_loss: 2563.1064
Epoch 910/

Epoch 976/1000
41/41 [==============================] - 0s 120us/step - loss: 2226.7104 - val_loss: 1967.0535
Epoch 977/1000
41/41 [==============================] - 0s 127us/step - loss: 2276.1772 - val_loss: 2010.8068
Epoch 978/1000
41/41 [==============================] - 0s 145us/step - loss: 2278.0613 - val_loss: 2104.3931
Epoch 979/1000
41/41 [==============================] - 0s 151us/step - loss: 2286.0269 - val_loss: 1770.2415
Epoch 980/1000
41/41 [==============================] - 0s 127us/step - loss: 2219.8455 - val_loss: 2299.3704
Epoch 981/1000
41/41 [==============================] - 0s 138us/step - loss: 2221.8970 - val_loss: 1400.7097
Epoch 982/1000
41/41 [==============================] - 0s 125us/step - loss: 2312.2505 - val_loss: 2925.3279
Epoch 983/1000
41/41 [==============================] - 0s 126us/step - loss: 2457.1370 - val_loss: 1458.5089
Epoch 984/1000
41/41 [==============================] - 0s 128us/step - loss: 2623.7952 - val_loss: 2716.7253
Epoch 985/

## 產生 Predict 2019/04/02 ～ 2019/04/08 所需的Testing feature
取 2018/04/02 ～ 2018/04/08 以及網站上的 2019/04/02前一週的資料
但沒有2019/04/01所以用2018/04/01代替

In [23]:
def getTestX(tp_2018):
    findDate = 0
    index = 0
    test_list = []
    for d in tp_2018:
        if d[0] == '20180402':
            findDate = 1
            index += 1
            test_list += d[1:]
        if findDate is 1:
            index += 1
            test_list += d[1:]
            #test_list.append(d)
        if index is 7:
            break
    return test_list

In [24]:
testList = getTestX(tp_2018)
test_2019_list = ['28756', '1887', '6.56', '29140', '1933', '6.63', '30093', '1892', '6.29', '29673', '2054', '6.92', '25810', '2155', '8.35', '24466', '2298', '9.39', '23895', '1655', '6.92']
test_X = testList + test_2019_list
np_test_X = np.array(test_X)
#print(test_X)
#print(len(test_X))

In [25]:
#print(np_test_X.shape)
test = np_test_X.reshape(1, 42)
#print(test.shape)
test_Y = LinearModel.predict(test)
print(test_Y[0])

[27154.893 30120.434 26721.951 27795.176 27002.11  25963.646 22157.56 ]


In [26]:
def saveCSV(test_Y):
    date = 20190402
    d_list = []
    temp_list = []
    index = 0
    for t in test_Y:
        temp_list.append(str(date+index))
        temp_list.append(t)
        d_list.append(temp_list)
        temp_list = []
        index += 1
    df = pd.DataFrame(d_list, columns=['date', 'peak_load(MW)'])
    return df

In [27]:
df = saveCSV(test_Y[0])
df.to_csv('submission.csv', index=0)

2019.04.01
P76071200
資工所 馬崇堯